# Workshop 3W — Upsell Like an Engineer
Unit 3 — Programming for Intelligent Products

Today we’ll practise TDD (Red → Green → Refactor), experiment logging, and debugging discipline.

In [73]:
import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
import mlflow, pandas as pd
print("MLflow:", mlflow.__version__)
print("Environment ready ✅")

Python: 3.12.1 (main, Nov 27 2025, 10:47:52) [GCC 13.3.0]
Platform: Linux-6.8.0-1030-azure-x86_64-with-glibc2.39
MLflow: 2.15.1
Environment ready ✅


In [74]:
def should_upsell(order: dict) -> bool:
    """
    Predicts if a customer should be upsold a drink using defensive 
    programming to handle messy input data.
    """
    
    # --- 1. Data Normalization (Cleaning) ---
    # Ensure time and loyalty are lowercase strings, even if None or mixed case
    time = str(order.get("time_of_day", "")).strip().lower()
    loyalty = str(order.get("loyalty_member", "no")).strip().lower()
    
    # Ensure temp and size are integers; handle cases where value might be None
    temp = int(order.get("temperature") or 0)
    size = int(order.get("order_size") or 0)

    # --- 2. The Logic (Priority Based) ---

    # RULE 1: The "Busy Lunch" Veto (Test 5)
    # If it's a heatwave at lunch but they aren't loyalty, we block the upsell.
    # Note: This MUST come before the general heatwave check.
    if temp >= 30 and time == "lunch" and loyalty != "yes":
        return False

    # RULE 2: General Heatwave (Test 3)
    # Upsell if it's hot (30 degrees or higher)
    if temp >= 30:
        return True

    # RULE 3: Large Order (Test 4)
    # Upsell if they are ordering for a group
    if size >= 4:
        return True

    # RULE 4: Loyalty at Lunch (Test 1a)
    # The standard marketing rule
    if loyalty == "yes" and time == "lunch":
        return True

    # DEFAULT: If no rules matched, don't upsell
    return False

In [75]:
# Positive test – loyalty at lunch → True
order_1a = {"time_of_day": "lunch", "loyalty_member": "yes"}
assert should_upsell(order_1a) is True, "FAIL: Loyalty at lunch should be upsold."
print("PASS: Test 1a")

# Negative test – non-loyalty at lunch → False (prevents `return True`)
order_1b = {"time_of_day": "lunch", "loyalty_member": "no"}
assert should_upsell(order_1b) is False, "FAIL: Non-loyalty at lunch should NOT be upsold."
print("PASS: Test 1b")

PASS: Test 1a
PASS: Test 1b


In [76]:
# Evening loyalty → should NOT be upsold
order_2 = {"time_of_day": "evening", "loyalty_member": "yes"}
assert should_upsell(order_2) is False, "FAIL: Evening loyalty should NOT be upsold."
print("PASS: Test 2")

PASS: Test 2


In [77]:
# Heatwave (temperature 30 or greater) → True
order_3 = {"time_of_day": "evening", "loyalty_member": "no", "temperature": 32}
assert should_upsell(order_3) is True, "FAIL: Hot weather should trigger upsell."
print("PASS: Test 3")

# Large order (order size 4 or greater) → True
order_4 = {"order_size": 4}
assert should_upsell(order_4) is True, "FAIL: Large orders should trigger upsell."
print("PASS: Test 4")

PASS: Test 3
PASS: Test 4


In [78]:
# Busy lunch heatwave (non-loyalty) → should NOT upsell
order_5 = {"time_of_day": "lunch", "loyalty_member": "no", "temperature": 32}
assert should_upsell(order_5) is False, "FAIL: Busy lunch heatwave (non-loyalty) should NOT upsell."
print("PASS: Test 5")

PASS: Test 5


In [79]:
try:
    import mlflow
    mlflow.set_tracking_uri("file:./mlruns")
    mlflow.set_experiment("hotdog-upsell")
    with mlflow.start_run(run_name="Initial Loyalty Rule"):
        order = {"time_of_day": "lunch", "loyalty_member": "yes"}
        prediction = should_upsell(order)
        mlflow.log_params(order)
        mlflow.log_metric("prediction", int(bool(prediction)))
        mlflow.set_tag("rule_version", "v1")
    print("Experiment logged successfully ✅")
except Exception:
    print("[INFO] MLflow not installed — skipping logging safely.")

Experiment logged successfully ✅


In [80]:
print("### My Commit Log")
commits = [
    "feat: Add initial failing test for lunch loyalty",
    "fix: Implement lunch loyalty rule",
    "feat: Add evening loyalty test",
    "fix: Handle time-of-day logic",
    "refactor: Simplify function and add docstring",
    "fix: Add busy heatwave override",
]
for c in commits:
    print("-", c)

### My Commit Log
- feat: Add initial failing test for lunch loyalty
- fix: Implement lunch loyalty rule
- feat: Add evening loyalty test
- fix: Handle time-of-day logic
- refactor: Simplify function and add docstring
- fix: Add busy heatwave override
